Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

SPDX-License-Identifier: Apache-2.0

# Prepare WiFi Data for GDN
GDN is an unsupervised anomaly detection algorithm that identifies anomalies at a timestep level for an entire system. This system consists of nodes that generate time-series (such as sensors in a water treatment plant), and GDN learns the relationship between the nodes during non-anomalous operation. These learned relationships can then be used at inference time to identify if the system is operating with anomalies.

To use GDN, we need a text file containing the list of nodes (will be referred to as 'sensors'), a train CSV containing timesteps and time series with for each sensor, and a test CSV containing timesteps and time series for each sensor, and labels for each time step.


This notebook prepares the wifi data and constructs the necessary train, test, and list.txt files for GDN

## Table of Contents  

[Imports](#imports)   
[Understanding the Data](#understand-data)   
[Setup](#setup)   
[Data Cleaning](#data-cleaning)   
[Exploratory Data Analysis of Cleaned Data](#EDA)   
[Assigning "Attack" Labels](#labels)   
[Normalizing Data](#normalize-data)
[Saving Cleaned Data for GDN](#saving-data)   
[References](#references)

## Imports <a name="imports"></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
import pathlib
from sklearn.preprocessing import MinMaxScaler

## Understanding the Data <a name="understand-data"></a>
 
Despite the growing popularity of 802.11 wireless networks, users often suffer from connectivity problems and performance issues due to unstable radio conditions and dynamic user behavior among other reasons. Anomaly detection and distinction are in the thick of major challenges that network managers encounter. This dataset exploits simulation as an effective tool to setup a computationally tractable network. 

The data features are categorized in two main classes: Density Attributes and Usage Attributes.   
1. Density Attributes demonstrate how crowded is the place in terms of active attendant users, characterizing the association population and durability,
2. Usage Attributes disclose the volume of the sent and received traffics by the present users, revealing the total bandwidth throughput regardless of how populous is the place and it is more relevant to the applications utilized by the current mobile users.

We take a look at the usage attributes for subpopulation Net 1 and Net2. Net1 represents a subpopulation with normal behavior, and Net2 contains anomalies. We try to use graph based anomaly detection approaches to accurately identify the anomalies in Net2. 


## Setup  <a name="setup"></a>

GDN requires the following files:     
            1. **list.txt:** the feature names, one feature per line  
            2. **train.csv:** training data modeling normal behavior, no anomalies were present according to the paper  
            3. **test.csv:** test data.test.csv should have a column named "attack" which contains ground truth label (0/1) of being attacked or not (0: normal, 1:
            attacked)  

This notebook creates the 3 files mentioned above and will save them in `../../data/03_primary/wifi`

## Data Cleaning

Currently, in each Net, each subpopulation contains a UDP directory, containing a sent and received csv for each of the 100 users. Currently, the raw data has a time column and a column formatted `Net2.cliHostx[y].udp.sentPk:vector(packetBytes)` where `y` represents the host. We want to clean the column names to only include the host name and round the time values to the nearest second. This is an example of one of the tables: 


| time      | Net2.cliHostx[y].udp.sentPk:vector(packetBytes) |
| ----------- | ----------- |
| 1.15      | 100       |
| 63.2   | 231        |


In order to run any graph based anomaly detection technique, we need a complete time series dataset. the UDP data is incomplete. Each file represents a user, who is either sending or receiving packets. Per subpopulation, there are 100 users, both sending and receiving packets -- 200 csv files. Within each of these files, we have a time column and a value columm, representing number of packets (in bytes) sent/received.

We to fill in the missing time values with 0's since no bytes were either sent or received at that time. This will give us a more complete time series dataset. We want to sum the values for each host. For example, if host 8 received 10 bytes from user 3 and 12 bytes from user 4 at time t, then the value for time t should be 22. Finally, we want to concatenate the datasets to have one large dataset, with the time as index, and the hosts as column names or sensors. We need both a test and a train dataset for GDN. The train data will be from Net1 (normal behavior) and the test data will come from Net 2 (containing anomalies)

In [ ]:
def clean_host_data(udp_subpopulation_data_path, sent=False):
    all_nums = []
    host_user_map = {}
    maximum_time = get_maximum_time(udp_subpopulation_data_path)
    all_data = []
    filename_prefix_check = 'r'
    if sent:
        filename_prefix_check = 's'
        
    for csv in os.listdir(net2_sub_0_path):
        if csv[0]==filename_prefix_check:
            try: 
                df = pd.read_csv(udp_subpopulation_data_path + csv)
                host_column = df.columns[1]
                host = int(host_column.split("[")[1].split("]")[0])
                df.set_axis(['time', str(host)], axis=1, inplace=True)

                if host not in host_user_map:
                    host_user_map[host] = []
                host_user_map[host].append(csv)
                time_value_map = contruct_time_value_map(df)
                column_values = fill_time_gaps(df, time_value_map, maximum_time)
                df = impute_data(df, maximum_time, column_values)
                all_data.append(df)
            except:
                print("cannot load file: ", csv)
                pass
    return all_data
        
def get_maximum_time(udp_subpopulation_data_path):
    maximum_time = 0
    for csv in os.listdir(udp_subpopulation_data_path):
        try:
            df = pd.read_csv(udp_subpopulation_data_path + csv)
            df = df.round({"time":0})
            maximum_time =max(max(df["time"]), maximum_time)
        except:
            print("cannot load file:   ", csv)    
    return maximum_time

def contruct_time_value_map(df):
    host = df.columns[1]
    time_value_map = {}
    for index, row in df.iterrows():
        time_value_map[int(row["time"])] = row[host]
    return time_value_map

def fill_time_gaps(df, time_value_map, maximum_time):
    column_values = []
    for t in range(int(maximum_time)):
        if t+1 in time_value_map:
            column_values.append(time_value_map[t+1])
        else:
            column_values.append(0)
    return column_values

def impute_data(df, maximum_time, column_values):
    time = [i+1for i in range(int(maximum_time))]
    host = df.columns[1]
    df = pd.DataFrame(data={"time": [i+1for i in range(int(maximum_time))], host: column_values})
    return df

def reconstruct_dataset(maximum_time, udp_subpopulation_data_path, output_path=""):
    print("here")
    for csv in os.listdir(udp_subpopulation_data_path):
        try:
            df = pd.read_csv(udp_subpopulation_data_path + csv)
            time_value_map = contruct_time_value_map(df)
            column_values = fill_time_gaps(df, time_value_map, maximum_time)
            df = impute_data(df, maximum_time, column_values)
            df.to_csv(output_path + csv, index=False)
        except:
            print("cannot load file:   ", csv)
            

def construct_dataset_aggregate_hosts(all_host_data):
    data = {}
    for df in all_host_data:
        host = df.columns[1]
        if host in data:
            aggregate_packets = data[host] + df[host]
            data[host] = aggregate_packets
        else:
            data[host] = df[host]

    return pd.DataFrame(data=data)
            

In [ ]:
# Net2 sent and received dataframes
net2_sub_0_path = "../../data/01_raw/wifi/wifi_data/Net2/0/UDP/"
net2_cleaned_sent_dfs = clean_host_data(net2_sub_0_path, sent=True)
net2_cleaned_rcvd_dfs = clean_host_data(net2_sub_0_path, sent=False)

In [ ]:
net2_cleaned_sent_dfs[0].columns

In [ ]:
# Net1 sent and received dataframes
net1_sub_0_path ="../../data/01_raw/wifi/wifi_data/Net1/0/UDP/"
net1_cleaned_sent_dfs = clean_host_data(net1_sub_0_path, sent=True)
net1_cleaned_rcvd_dfs = clean_host_data(net1_sub_0_path, sent=False)

In [ ]:
net1_cleaned_sent_dfs[0].columns

We summed packets from the same hosts. We must do this for both sent and received files. First we aggregate the received (rcvd) files then the sent files. We will then get the sent - received values for each host.

In [ ]:
net1_sent_aggregated_df = construct_dataset_aggregate_hosts(net1_cleaned_sent_dfs)
net1_sent_aggregated_df.head()

In [ ]:
net1_rcvd_aggregated_df = construct_dataset_aggregate_hosts(net1_cleaned_rcvd_dfs)
net1_rcvd_aggregated_df.head()

In [ ]:
net2_sent_aggregated_df = construct_dataset_aggregate_hosts(net2_cleaned_sent_dfs)
net2_rcvd_aggregated_df = construct_dataset_aggregate_hosts(net2_cleaned_rcvd_dfs)

net2_sent_aggregated_df.head()

In [ ]:
net2_rcvd_aggregated_df.head()

## Exploratory Data Analysis of Cleaned Data <a name="EDA"></a>

In [ ]:
net1_sent_df_hosts = set(net1_sent_aggregated_df.columns)
net1_rcvd_df_hosts  = set(net1_rcvd_aggregated_df.columns)
net1_rcvd_df_hosts == net1_sent_df_hosts

In [ ]:
net2_sent_df_hosts = set(net2_sent_aggregated_df.columns)
net2_rcvd_df_hosts  = set(net2_rcvd_aggregated_df.columns)
net2_rcvd_df_hosts == net2_sent_df_hosts

#verifying the hosts are the same
net2_rcvd_df_hosts == net2_sent_df_hosts

We subtract the received bytes from the sent bytes. Ideally, the same number of packets sent, should also be received. Understanding the number of leftover packets will help us understand the data and aggregate it into one dataset in a meaningful way. 

In [ ]:
def subtract_received_from_sent(sent_df, rcvd_df):
    sent_minus_received = {}
    for host in sent_df:
        sent_minus_received[host]=sent_df[host] - rcvd_df[host]
    df = pd.DataFrame(data=sent_minus_received)
    return df

In [ ]:
net1_subtracted_data = subtract_received_from_sent(net1_sent_aggregated_df, net1_rcvd_aggregated_df)
net2_subtracted_data = subtract_received_from_sent(net2_sent_aggregated_df, net2_rcvd_aggregated_df)

In [ ]:
net2_subtracted_data.head()

In [ ]:
net1_subtracted_data.head()

In [ ]:
column_names = net1_subtracted_data.columns
column_names

In [ ]:
# Keeping only the first 1200 rows, 1200 seconds so the normal behavior and anomaly data are the same length
net1_subtracted_data=net1_subtracted_data.head(1200)
len(net1_subtracted_data)

In [ ]:
len(net2_subtracted_data)

Plotting number of packets per host, overlaying the anomaly data and the normal data to see if we can visualize anomalies

In [ ]:
def plot_num_packets(net1, net2, host):
    fig = plt.figure(figsize=(15, 10))
    time = [i for i in range(1200)]
    plt.scatter(time, net2[host], alpha = 0.3, label="Anomaly Net2")
    plt.scatter(time, net1[host], alpha=0.3, color = "orange", label= "Normal Behavior Net1")
    title = "Number of Packets Host: " + host
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Number of Packets Sent- Received ")
    plt.legend()

In [ ]:
for host in column_names[:4]:
    plot_num_packets(net1_subtracted_data, net2_subtracted_data, host)

## Assigning "Attack" Labels <a name="labels"></a>

In [ ]:
net2_subtracted_data.columns

In [ ]:
net2_subtracted_data.head(n=5)

To assign attack labels, we will subtract the packets for the normal and anomaly data.  
0: [2*Lower Quartile, 2*Upper Quartile]  
1: otherwise  

In [ ]:
def assign_attack_labels(normal_data, 
                         anomaly_data):
    all_data = {}
    for host in normal_data.columns:
        all_data[host] =  abs(normal_data[host]-anomaly_data[host])
    all_data = pd.DataFrame(all_data)
    
    
    all_host_stats = {}
    for host in all_data.columns:
        host_data = dict(all_data[host].describe())
        all_host_stats[host]=host_data
        
    all_host_stats
    anomaly_times = set()
    for host in all_data.columns:
        min_accepted_value = -2*all_host_stats[host]["25%"]
        max_accepted_value =  2*all_host_stats[host]["75%"]
        for index, row in all_data.iterrows():
            value = row[host]
            if (value < min_accepted_value or value > max_accepted_value) and (int(value) != 0):
                anomaly_times.add(index)
                
    anomaly_data["attack"] = [0 for i in range(len(anomaly_data))]
    for index, row in anomaly_data.iterrows():
        if index in anomaly_times:
            anomaly_data["attack"][index]= 1
    return anomaly_data, anomaly_times


In [ ]:
net2_with_labels, anomaly_times = assign_attack_labels(net1_subtracted_data, net2_subtracted_data)

In [ ]:
net2_with_labels

In [ ]:
# Notice how there are no zeros in the anomaly times
print(sorted(list(anomaly_times)))

In [ ]:
# Fraction of data classified as anomalies
len(anomaly_times)/1200

In [ ]:
# Verifying the attack column has been populated properly. Recall, there were 400 anomalies identified from our method above
sum(net2_with_labels["attack"])

In [ ]:
net2_with_labels.columns

In [ ]:
net2_with_labels.head(4)

Plotting the number of packets sent - received. This way we can see where the anomalies actually lie. 

In [ ]:
def plot_num_packets_overlay_attack(normal, anomaly, host):
    fig = plt.figure(figsize=(15, 10))
    plt.scatter(anomaly.index, anomaly[host], alpha = 0.3, label="Anomaly")
    plt.scatter(normal.index, normal[host], alpha=0.3, color = "orange", label= "Normal Behavior")
    plt.scatter(normal.index, anomaly["attack"], alpha=0.3, color = "red", label= "Classified Anomaly")
    title = "Number of Packets with Highlighted Anomalies Host:  " + host
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Number of Packets Sent-Received")
    plt.legend()

In [ ]:
column_names = net2_with_labels.columns[:-1]
column_names

In [ ]:
for host in column_names[:4]:
    plot_num_packets_overlay_attack(net1_subtracted_data, net2_with_labels, host)

In [ ]:
net1_subtracted_data

In [ ]:
net2_with_labels

## Normalizing Data

In [ ]:
# max min(0-1)
def norm(train, test):
    normalizer = MinMaxScaler(feature_range=(0, 1)).fit(train) # scale training data to [0,1] range
    train_ret = normalizer.transform(train)
    test_ret = normalizer.transform(test)

    return train_ret, test_ret

In [ ]:
test = net2_with_labels
train = net1_subtracted_data
attack_column = test["attack"]

test = test.iloc[:, 1:]
train = train.iloc[:, 1:]

train = train.fillna(train.mean())
test = test.fillna(test.mean())
train = train.fillna(0)
test = test.fillna(0)

train_columns = train.columns
test_columns = test.columns

# trim column names
train = train.rename(columns=lambda x: x.strip())
test = test.rename(columns=lambda x: x.strip())

print(len(test.columns),test.columns)
print(len(train.columns),train.columns)


# train_labels = train.attack
test_labels = test.attack

# train = train.drop(columns=['attack'])
test = test.drop(columns=['attack'])


x_train, x_test = norm(train.values, test.values)

In [ ]:
train_df = pd.DataFrame(x_train, columns = train_columns)
test_df =pd.DataFrame(x_test, columns = test_columns[:-1])
train_df.head()

## Saving Cleaned Data for GDN   <a name="saving-data"></a>

In [ ]:
test_df["attack"] = attack_column

In [ ]:
def descriptive_column_names(df):
    new_column_names = {}
    for column in df.columns:
        if column != "attack":
            new_column_names[column] = "host_"+column
    return new_column_names

In [ ]:
train_df.rename(columns=descriptive_column_names(train_df), inplace=True)
test_df.rename(columns=descriptive_column_names(test_df), inplace=True)

In [ ]:
test_df.columns

In [ ]:
train_df.columns

In [ ]:
pathlib.Path("../../data/03_primary/wifi/gdn").mkdir(parents=True, exist_ok=True)

In [ ]:
train_df.to_csv("../../data/03_primary/wifi/gdn/wifi_gdn_train.csv", index=False)
test_df.to_csv("../../data/03_primary/wifi/gdn/wifi_gdn_test.csv", index=False)

with open("../../data/03_primary/wifi/gdn/wifi_sensor_list.txt", "w") as f:
    f.writelines("\n".join(train_df.columns))

# References <a name="references"></a>

Anisa Allahdadi and Ricardo Morla. 2017. 802.11 Wireless Access Point Usage Simulation and Anomaly Detection. CoRR abs/1707.02933, (2017). Retrieved from http://arxiv.org/abs/1707.02933 

Ailin Deng and Bryan Hooi. 2021. Graph Neural Network-Based Anomaly Detection in Multivariate Time Series. CoRR abs/2106.06947, (2021). Retrieved from https://arxiv.org/abs/2106.06947 